# Pre-processing

### Loading necessary packages

In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

### Loading each school

In [2]:
man = pd.read_csv('manhattan_cc.csv')
nas = pd.read_csv('nassau_cc.csv')
lg = pd.read_csv('laguardia_cc.csv')
qns = pd.read_csv('queensboro_cc.csv')
king = pd.read_csv('kingsboro_cc.csv')

### removing duplicated rows and then resetting the index

In [3]:
man = man.drop_duplicates().reset_index(drop = True)
nas = nas.drop_duplicates().reset_index(drop = True)
lg = lg.drop_duplicates().reset_index(drop = True)
qns = qns.drop_duplicates().reset_index(drop = True)
king = king.drop_duplicates().reset_index(drop = True)

### adding "school" column to each df

In [4]:
man["school"] = "Borough of Manhattan CC"
nas["school"] = "Nassau CC"
lg["school"] = "LaGuardia CC"
qns["school"] = "Queensboro CC"
king["school"] = "Kingsboro CC"

### copying each df

In [5]:
man2 = man.copy()
nas2 = nas.copy()
lg2 = lg.copy()
qns2 = qns.copy()
king2 = king.copy()

### creating master df

In [6]:
total = pd.concat([man2, nas2, lg2, qns2, king2])

### keeping only some columns, removing duplicates, resetting index

In [7]:
total = total[["name", "department", "overall_score", \
               "difficulty_score", "chili", "tag_list", "content", "school"]].dropna().reset_index(drop = True)

# Processing the data

### reviews don't include information on sex, so I use the comments to determine

#### creating a list of male and female pronouns

In [8]:
male_words = ["he", "him", "his", "he's"]
female_words = ["she", "her", "hers", "she's"]

### Creating two new columns to the df: 
### 1) the num of male pronouns and 2) the num of female pronouns for each row
* 1) copy data set
* 2) create empty lists for number of male and female words
* 3) iterating through each row the entire data set
* 4) creating empty values used to enumerate number of male and female words for each row
* 5) removing punctuation, changing all to lower case, making a list of all words
* 6) iterating through all words of the list to and if
* 7) it matches a word in ```male_words```, add 1 to ```male```
* 8) it matches a word in ```female_words```, add 1 to ```female```
* 9) add ```male``` and ```female``` to ```male_column_list``` and ```female_column_list```
* 10) add each of these lists to the total df as new columns

In [9]:
total2 = total.copy() # 1
male_column_list = [] # 2
female_column_list = [] # 2
for i in range(len(total2)): # 3
    male = 0 # 4
    female = 0 # 4
    word_list = re.sub(r'[^\w\s]','', total2["content"][i]).lower().split(" ") # 5
    for word in word_list: # 6
        if word in male_words: #7
            male += 1
        if word in female_words: #8
            female += 1
    male_column_list.append(male) # 9
    female_column_list.append(female)
    
total2["num_male_words"] = male_column_list # 10
total2["num_female_words"] = female_column_list

### Now we have the number of male and female pronoun names in each row, but we need to add them all up for each professor to see if there are more male or female pronouns
* 1) grouping the data by professor name and school
* 2) calculating the sum for male and female pronouns for each professor
* 3) the groupby function changes the index, so here it is reset
* 4) creating a new df of only the columns I want to merge back with the original data
* 5) merging by name and school
* 6) creating a new column, ```sex_diff```, determining the difference in ```male``` and ```female```
* 7) based on the sign of ```sex_diff```, estimate the sex of the professor
* 8) reorder columns (and drop some)

In [10]:
total2_group = total2.groupby(["name", "school"]) # 1
total2_group = total2_group["num_male_words", "num_female_words"].sum() # 2
total2_group = total2_group.reset_index() # 3
total3_group = total2_group[["name", "school", "num_male_words", "num_female_words"]] # 4
total_merge = pd.merge(total2, total3_group, how = "left", on = ('name', 'school'), suffixes = ("", "_total")) # 5
total_merge["sex_diff"] = total_merge["num_male_words_total"] - total_merge["num_female_words_total"] # 6
total_merge["sex"] = np.where(total_merge["sex_diff"] > 0, "male", "female") # 7
total_merge = total_merge[["name", "sex", "school", "department", "overall_score", \
                           "difficulty_score", "chili", "tag_list", "content"]] # 8

### the "department" column is too long and includes unnecessary info, so I replace the column with just the name of the department

In [11]:
total_merge['department'] = total_merge['department'].replace({r' department': '', r'Professor in the ': ''}, regex = True)

### different colleges name similar departments in different ways, or professors are teaching classes outside of their home department so here I concatenate similar departments and reassign professors to their departments

In [12]:
total_merge.loc[total_merge['department'].str.contains('Biological'), 'department'] = 'Biology'
total_merge.loc[total_merge['department'].str.contains('Business Te'), 'department'] = 'Business'
total_merge.loc[total_merge['department'] == 'Science', 'department'] = 'Physical Sciences'
total_merge.loc[total_merge['department'].str.contains('Theater|Speech'), 'department'] = 'Communication'
total_merge.loc[total_merge['name'] == 'Greco, Joseph', 'department'] = 'Communication'
total_merge.loc[total_merge['name'] == 'Sokolski, Patricia', 'department'] = 'Communication'
total_merge.loc[total_merge['department'].str.contains('Allied'), 'department'] = 'Health Science'
total_merge.loc[total_merge['department'].str.contains('Health & Physical Education|Physical Education'), 'department'] = 'Physical Ed'
total_merge.loc[total_merge['department'].str.contains('Art'), 'department'] = 'Art'
total_merge.loc[total_merge['department'].str.contains('Legal'), 'department'] = 'Law'
total_merge.loc[total_merge['department'].str.contains('African|Ethnic|Women'), 'department'] = 'Cultural Studies'
total_merge.loc[total_merge['department'].str.contains('Foreign|Spanish'), 'department'] = 'Languages'
total_merge.loc[total_merge['department'].str.contains('Info'), 'department'] = 'Social Science'
total_merge.loc[total_merge['name'] == 'Townsend, Charles', 'department'] = 'Social Science'
total_merge.loc[total_merge['name'] == 'Ruiz, Roberto', 'department'] = 'Philosophy'
total_merge.loc[total_merge['department'].str.contains('History|Political|Philosophy'), 'department'] = 'History, Philosophy, Poly Sci'

### students can give professors a "chili pepper" icon to denote if they are attractive or not, here I create a new column adding whether the prof received a chili pepper

In [13]:
total_merge['chili'] = np.where(total_merge['chili'] == True, 'attractive', 'not attractive')

### For each review, a reviewer can add up to 3 (out of 20) different "tags" describing the professor. Here I group the reviews by professor, then create a dictionary of the tags of each review for that professor,  then make each tag a column in the original data frame which shows how many of each of the 20 available tags each professor received

### the goal is to find the proportion of each tag type a professor received

* 1) group df by professor name
* 2) create empty list for all tag dictionaries
* 3) groupy objects behave similarly to dictionaries; name is the key, and the corresponding df is the value, so I iterate through all dfs grouped by professor name
* 4) create empty list that will include all reviews' tags
* 5) each review's tag list is scraped in as a string, and if a student did not add any tags, then a string of an empty list is left (i.e., ```"[]"```), so
* 6) I remove all strings that only contain ```"[]"```
* 7) and add all the remaining strings of tags ```tags```
* 8) create empty string
* 9) because each set of tags is a string, I concatenate all tags for a single prof into one string
* 10) some tags are envoloped by single quotes ``` ' ``` and others in double quotes ``` " ```; here I make everything to be single quotes
* 11) replace all single quotes (which includes apostrophes) with nothing
* 12) replace the two types of punctuation in the tags (``` . ``` and ``` ? ```) with nothing
* 13) remove the final character of each string, which is ```]```
* 14) replace all ```[``` with nothing
* 15) replace all ```]``` with a comma and a space (to space all words) so that I can
* 16) split all the words by the comma, creating a list of tags
* 17) remove any whitespace from each tag in the list
* 18) creating empty dictionary
* 19) adding a the name of the professor to the key of the dictionary
* 20) iterating through each tag of the list, and
* 21) if it is already present in the dictionary, add 1, else add it to the dictionary
* 22) add each professor's dictionary to the list of all dictionaries
* 23) converting the list of dictionaries to a df
* 24) an empty column was created, so here I drop it
* 25) merging the dictionary df back to the original df

In [14]:
profs_group = total_merge.groupby("name") # 1
total_tag_list = [] # 2
for key, values in profs_group: # 3
    tags = [] # 4
    for tag in values["tag_list"]: # 5
        if tag != "[]": # 6
            tags.append(tag.title().strip()) # 7

    tags_total = "" # 8
    for tag in tags: # 9
        tags_total += tag

    x = re.sub(r'"', "'", tags_total) # 10
    x = re.sub(r"'", "", x) # 11
    x = re.sub('[.?]', "", x) # 12
    x = x[:-1] # 13
    x = x.replace("[", "") # 14
    x = x.replace("]", ", ") # 15
    x = x.split(",") # 16
    x = [j.strip(" ") for j in x] # 17

    counts = dict() # 18
    counts["name"] = key # 19
    for i in x: # 20
        counts[i] = counts.get(i, 0) + 1 # 21    

    total_tag_list.append(counts) # 22
    
tag_dict_df = pd.DataFrame(total_tag_list) # 23
tag_dict_df.drop('', axis = 1, inplace = True) # 24
final = pd.merge(total_merge, tag_dict_df, how = "left", on = "name") # 25

### counting the number of tags for each professor

In [15]:
final['total_tags'] = final.iloc[:, 9:].sum(axis = 1)

### aggregating tag proportions

* 1) grouping by each professor, aggregating the mean scores, and reseting the index
* 2) renaming the scores to reflect that they are means
* 3) some professors received no tags, thus finding a proportion would result in a divide by zero error, so here I replace zeroes with ones
* 4) creating a list of columns whose proportions need to be aggregated
* 5) calculating the proportion of each tag

In [16]:
grouped = final.groupby(['name', 'school', 'sex', 'department']).mean().reset_index() # 1

grouped.rename(columns = {'overall_score':'overall_score_mean', 
                          'difficulty_score': 'difficulty_score_mean'}, inplace = True) # 2

grouped.loc[grouped['total_tags'] == 0, 'total_tags'] = 1 # 3

tag_names = grouped.columns[6:-1] # 4
grouped[tag_names] = grouped[tag_names].apply(lambda x: x / grouped['total_tags'] * 100) # 5

### Saving the datasets for use in the R Shiny app

#### one data set with all reviews (where each professor's review is a different row)
#### one data set where each row is a professor with his/her ratings and proportions aggregated

In [17]:
final.head()

,name,sex,school,department,overall_score,difficulty_score,chili,tag_list,content,Accessible Outside Class,Amazing Lectures,Beware Of Pop Quizzes,Caring,Clear Grading Criteria,Extra Credit,Get Ready To Read,Gives Good Feedback,Graded By Few Things,Group Projects,Hilarious,Inspirational,Lecture Heavy,Lots Of Homework,Participation Matters,Respected,Skip Class You WonT Pass,So Many Papers,Test Heavy,Tough Grader,total_tags
0,"Beran, Andrew",male,Borough of Manhattan CC,Mathematics,5.0,2.0,attractive,"['AMAZING LECTURES', 'RESPECTED', 'CLEAR GRADI...","Best professor ever, I do not usually write re...",NaN,42.0,7.0,38.0,50.0,NaN,1.0,17.0,3.0,NaN,82.0,32.0,NaN,NaN,3.0,60.0,19.0,NaN,NaN,2.0,356.0
1,"Beran, Andrew",male,Borough of Manhattan CC,Mathematics,5.0,2.0,attractive,"['HILARIOUS', 'INSPIRATIONAL', 'CARING']",Professor Beran is the most amazing math teach...,NaN,42.0,7.0,38.0,50.0,NaN,1.0,17.0,3.0,NaN,82.0,32.0,NaN,NaN,3.0,60.0,19.0,NaN,NaN,2.0,356.0
2,"Beran, Andrew",male,Borough of Manhattan CC,Mathematics,5.0,1.0,attractive,"[""SKIP CLASS? YOU WON'T PASS."", 'HILARIOUS', '...",THE BERANSTER is alive! Professor Beran is ver...,NaN,42.0,7.0,38.0,50.0,NaN,1.0,17.0,3.0,NaN,82.0,32.0,NaN,NaN,3.0,60.0,19.0,NaN,NaN,2.0,356.0
3,"Beran, Andrew",male,Borough of Manhattan CC,Mathematics,4.0,2.0,attractive,[],professor beran is an amazing professor he exp...,NaN,42.0,7.0,38.0,50.0,NaN,1.0,17.0,3.0,NaN,82.0,32.0,NaN,NaN,3.0,60.0,19.0,NaN,NaN,2.0,356.0
4,"Beran, Andrew",male,Borough of Manhattan CC,Mathematics,5.0,3.0,attractive,"['GIVES GOOD FEEDBACK', 'RESPECTED', 'CLEAR GR...",Professor Andrew is a very talented man. You c...,NaN,42.0,7.0,38.0,50.0,NaN,1.0,17.0,3.0,NaN,82.0,32.0,NaN,NaN,3.0,60.0,19.0,NaN,NaN,2.0,356.0


In [18]:
grouped.head()

,name,school,sex,department,overall_score_mean,difficulty_score_mean,Accessible Outside Class,Amazing Lectures,Beware Of Pop Quizzes,Caring,Clear Grading Criteria,Extra Credit,Get Ready To Read,Gives Good Feedback,Graded By Few Things,Group Projects,Hilarious,Inspirational,Lecture Heavy,Lots Of Homework,Participation Matters,Respected,Skip Class You WonT Pass,So Many Papers,Test Heavy,Tough Grader,total_tags
0,"Abdoo, Sherlyn",LaGuardia CC,female,English,2.564815,4.444444,NaN,NaN,NaN,NaN,NaN,NaN,7.692308,7.692308,NaN,7.692308,NaN,NaN,23.076923,NaN,NaN,NaN,7.692308,7.692308,NaN,38.461538,13.0
1,"Abdul Samad, Shofiyaa",LaGuardia CC,female,Business,4.637681,1.521739,NaN,4.255319,NaN,21.985816,4.964539,NaN,NaN,19.148936,1.418440,NaN,4.255319,16.312057,NaN,NaN,6.382979,18.439716,2.127660,NaN,NaN,0.709220,141.0
2,"Aceto, John",LaGuardia CC,male,English,4.284810,3.341772,2.325581,23.255814,NaN,NaN,2.325581,2.325581,13.953488,6.976744,NaN,NaN,23.255814,13.953488,NaN,NaN,4.651163,NaN,2.325581,NaN,NaN,4.651163,43.0
3,"Adamou, Nikolaos",Borough of Manhattan CC,male,Business,4.066667,2.658333,2.380952,9.523810,NaN,10.714286,2.380952,NaN,5.952381,5.952381,1.190476,NaN,4.761905,16.666667,1.190476,3.571429,5.952381,13.095238,10.714286,1.190476,1.190476,3.571429,84.0
4,"Adel, Kira",Nassau CC,female,Mathematics,4.844828,2.155172,1.694915,10.169492,NaN,20.338983,11.864407,NaN,NaN,22.033898,NaN,NaN,1.694915,5.084746,NaN,6.779661,NaN,11.864407,6.779661,NaN,1.694915,NaN,59.0


In [ ]:
final.to_csv("profs.csv")
grouped.to_csv("grouped_profs.csv")